In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
from HandTrackerRenderer import HandTrackerRenderer
from HandTrackerEdge import HandTracker
from FPS import FPS, now
from tensorflow import keras

In [34]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

In [5]:
DATA_PATH = os.path.join('Collected_Data') 
actions = [ action for action in os.listdir(DATA_PATH)]
print('The number of actions is',len(actions))
print(actions)
no_sequences =60
sequence_length = 30
start_folder = 30
label_map = {label:num for num, label in enumerate(actions)}

The number of actions is 29


In [8]:
sign_model = keras.models.load_model('action29_LSTM_TEST.h5')

In [9]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [10]:
def extractData(hands) :
    rh_data = np.zeros((21, 3))
    lh_data = np.zeros((21, 3))
    for hand in hands : 
        Label = hand.label
        if Label == 'left' : 
            lh_data = hand.norm_landmarks 
        elif Label == 'right' : 
            rh_data = hand.norm_landmarks
    return lh_data,rh_data

In [ ]:
nouns = ['shirt', 'skirt']
pronouns = ['it','you']
adjectives = ['pretty', 'bad']
verbs = ['am','are','do','have','is','sell','wear','look','think']
adverbs = []
prepositions = ['to','about']
conjunctions = ['so']
articles = ['a','the']
personals = ['I']
numbers = ['one','two','three','four','five','six']
checklist = 0
stop = False

def gramcheck(predicted_sign,now_sign) :
    global checklist
    global stop
    if len(now_sign) != 0 :
        now_sign = now_sign[0]
    if now_sign == [] :
        return True
    if (now_sign in personals) and (predicted_sign in verbs) :
        checklist+=1
        return True
    if (now_sign in nouns) and (predicted_sign in verbs)  :  
        checklist+=1
        return True
    if (now_sign in pronouns) and (predicted_sign in verbs) :
        checklist+=1
        return True
    if (now_sign in adjectives) and (predicted_sign in adjectives or predicted_sign in nouns) :
        if checklist >= 2 :
            checklist = 0
            stop = True
            return '.'
        else :  
            checklist+=1
            return True 
    if (now_sign in verbs) and (predicted_sign in articles or predicted_sign in adjectives or predicted_sign in numbers) :
        if predicted_sign in articles or predicted_sign in numbers :
            checklist+=1
            return True
        if predicted_sign in adjectives :
            if checklist >= 1 :
                checklist = 0
                stop = True
                return '.'
            else :
                return False
    if (now_sign in articles) and (predicted_sign in nouns) :
        if checklist >= 2 :
            checklist = 0
            stop = True
            return '.'
        else :  
            checklist+=1
            return True
    if (now_sign in numbers) and (predicted_sign in nouns or predicted_sign in adjectives) :
        if predicted_sign in adjectives:
            checklist+=1
            return True
        if predicted_sign in nouns :
            if checklist >= 2 :
                checklist = 0
                stop = True
                return '.'
            else :
                checklist+=1
                return True
        else :
            return False
    else :
        return False
        

In [11]:
tracker = HandTracker(solo = False)

renderer = HandTrackerRenderer(
        tracker=tracker,
        output=None)

Palm detection blob     : C:\Secret\College\Senior\SIgnExactEnglish\models\palm_detection_sh4.blob
Landmark blob           : C:\Secret\College\Senior\SIgnExactEnglish\models\hand_landmark_lite_sh4.blob
PD post processing blob : C:\Secret\College\Senior\SIgnExactEnglish\custom_models\PDPostProcessing_top2_sh1.blob
Sensor resolution: (1920, 1080)
Internal camera FPS set to: 36
Internal camera image size: 1152 x 648 - pad_h: 252
Creating pipeline...
Creating Color Camera...
Creating Palm Detection pre processing image manip...
Creating Palm Detection Neural Network...
Creating Palm Detection post processing Neural Network...
Creating Hand Landmark pre processing image manip...
Creating Hand Landmark Neural Network (2 threads)...
Pipeline created.
Pipeline started - USB speed: SUPER


In [14]:
#Threshold = [are,for]
lh = []
rh = []
sentence = []
threshold = 0.8
now_sign=[]
font=cv2.FONT_HERSHEY_SIMPLEX
color=(0,255,0)
size = 1
thickness=2
while True:
    frame, hands, bag = tracker.next_frame()
    if frame is None: break
    # Draw hands
    hand = renderer.draw(frame, hands, bag)
    key = renderer.waitKey(delay=1)
    lh_data,rh_data = extractData(hands)
    lh.append(lh_data)
    rh.append(rh_data)
    lh_used=lh[-30:]
    rh_used=rh[-30:]
    lrh = np.concatenate([lh_used,rh_used])
    lrh_used = lrh.reshape(lrh.shape[0],-1)
    cv2.putText(frame,f"data = {len(lrh)}",(20,50),font,size,color,thickness)
    # predictions logic
    if len(lrh) == 60:
        res = sign_model.predict(np.expand_dims(lrh_used, axis=0))[0]
        predicted_sign = actions[np.argmax(res)]
        confidence = res[np.argmax(res)]
        print(predicted_sign,now_sign,checklist,stop)
        if (confidence > threshold) & (predicted_sign not in now_sign)  :
            if stop == True :
                print(predicted_sign)
                print(confidence)
                stop=False
                now_sign=[actions[np.argmax(res)]]
            if gramcheck(predicted_sign,now_sign) == True :
                print(predicted_sign)
                print(confidence)
                now_sign=[actions[np.argmax(res)]]
            elif gramcheck(predicted_sign,now_sign) == '.' :
                print(predicted_sign)
                print(confidence)
                now_sign=[actions[np.argmax(res)]]
        lh = []
        rh = []
    if key == 27 or key == ord('q'):
        break
renderer.exit()

wear
0.7681162
about
0.8830831
it
0.8303734
one
0.7432445
three
0.845091
four
0.7237945
five
0.9933634
six
0.7390419
wear
0.94746286
about
0.9925299
wear
0.8075761
about
0.99760604
for
0.97340536
you
0.99278694
look
0.9258108
four
0.8605087
Hello
0.96497405
bad
0.8679641
skirt
0.999534
do
0.9998969
skirt
0.9958788
you
0.9999809
skirt
0.9908173
about
0.9899809
do
1.0
it
1.0
skirt
0.9990048
to
0.96893936
do
0.9999865
about
0.9787409
skirt
0.7539312
Hello
0.7386446
skirt
0.97082204
it
1.0
sell
1.0
it
1.0
do
0.9998319
sell
0.9392219
do
0.9963981
you
0.99999213
wear
0.9845537
you
0.99974674
am
0.99954045
you
0.99998283
am
0.9999995
you
0.9999975
bad
0.807147
four
0.7957725
one
0.7946413
two
0.7610692
three
0.9085441
four
0.87903166
six
0.7251806
four
0.7419785
wear
0.7707841
I
0.7956868
am
0.991135
a
0.78880006
shirt
0.9601372
is
0.8954994
wear
0.847123
I
0.9351392
a
0.99761933
skirt
0.9968945
I
0.9090053
do
0.91138923
a
0.77831143
I
0.72363776
wear
0.8913606
a
0.9969078
skirt
0.9990528
abo

In [73]:
tracker.exit()